In [3]:
from datasets import load_dataset

In [2]:
#load reference dataset format that fit for llama-factory SFT trainer. 
data = load_dataset("c-s-ale/alpaca-gpt4-data-zh")
data['train'][0]

{'instruction': '保持健康的三个提示。',
 'input': '',
 'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'}

In [3]:
#load target dataset, aiming to convert its domain into reference dataset. 
dataset = load_dataset("medmcqa")

In [4]:
#convert num label into multi-choice label
def label_convert(dataset):
    return {'ans': chr(ord('A') + dataset['cop'])} 
dataset = dataset.map(label_convert)
dataset['train'][0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': 2,
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'ans': 'C'}

In [5]:
#check difference between target and reference data formats. config field_config functions. 
def field_config(dataset):
    return {'instruction': f"""{dataset['question']}: (A) {dataset['opa']}, (B) {dataset['opb']}, (C) {dataset['opc']}, (D) {dataset['opd']}""", 
            'input': f"""""",
            'output': f"""Answer is {dataset['ans']}. {dataset['exp']}"""}
dataset = dataset.map(field_config)

In [6]:
dataset = dataset.remove_columns(['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'ans'])
dataset['train'][0]

{'instruction': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma: (A) Hyperplasia, (B) Hyperophy, (C) Atrophy, (D) Dyplasia',
 'input': '',
 'output': 'Answer is C. Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950'}

In [65]:
from huggingface_hub import notebook_login; notebook_login()

In [8]:
dataset.push_to_hub("chenhugging/medmcqa_instruct")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/183 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chenhugging/medmcqa_instruct/commit/7c4a17dc1d4300698d3f117464774d16c99ab31d', commit_message='Upload dataset', commit_description='', oid='7c4a17dc1d4300698d3f117464774d16c99ab31d', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
from huggingface_hub import notebook_login; notebook_login()

In [54]:
#re-format OCN_ONCC_Practice_test_100 into medmcqa format

In [2]:
dataset = load_dataset('json', data_files='./OCN_ONCC_Practice_test_100.jsonl', split='train')
print(dataset[0])

{'question': 'Prior to receiving an initial dose of daunorubicin, the patient should first receive:', 'options': '{"A": an echocardiogram., "B": an abdominal ultrasound., "C": a urinalysis., "D": a lumbar puncture.}', 'answer': 'an echocardiogram.', 'answer_idx': 'A'}


In [58]:
data = load_dataset('json', data_files='./medmcqa_samples.jsonl', split='train')
print(data[0])

{'input': [{'role': 'system', 'content': 'You are a highly intelligent doctor who answers the following multiple choice question correctly.\nOnly write the answer down.'}, {'role': 'user', 'content': 'Subject: Anatomy\n\nChronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\n\na) Hyperplasia\nb) Hyperophy\nc) Atrophy\nd) Dyplasia'}], 'ideal': 'c) Atrophy'}


In [63]:
data

Dataset({
    features: ['input', 'ideal'],
    num_rows: 300
})

In [3]:
#check difference between target and reference data formats. config field_config functions. 
def field_config(dataset):
    in_1 = {'role': 'system', 'content': 'You are a highly intelligent doctor who answers the following multiple choice question correctly.\nOnly write the answer down.'}
    in_2 = {'role': 'user', 'content': f"""{dataset['question']} \n {dataset['options'].replace('{','').replace('}','')}""" }
    return {'input': [in_1, in_2], 
            'ideal': f"""{dataset['answer_idx']}"""
           }
dataset = dataset.map(field_config)

In [4]:
dataset = dataset.remove_columns(['question', 'options', 'answer', 'answer_idx'])

In [5]:
dataset

Dataset({
    features: ['input', 'ideal'],
    num_rows: 100
})

In [6]:
dataset[0]

{'input': [{'content': 'You are a highly intelligent doctor who answers the following multiple choice question correctly.\nOnly write the answer down.',
   'role': 'system'},
  {'content': 'Prior to receiving an initial dose of daunorubicin, the patient should first receive: \n "A": an echocardiogram., "B": an abdominal ultrasound., "C": a urinalysis., "D": a lumbar puncture.',
   'role': 'user'}],
 'ideal': 'A'}

In [ ]:
dataset.push_to_hub("chenhugging/ocn_oncc_practice_test")

In [7]:
dataset.save_to_disk("ocn_oncc_practice_test_hf.jsonl")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
import json

with open('output.jsonl', 'w') as outfile:
    for entry in dataset:
        json.dump(entry, outfile)
        outfile.write('\n')

In [10]:
dataset = load_dataset('json', data_files='./AOCNP_ONCC_Practice_test_100.jsonl', split='train')
print(dataset[0])

{'question': 'When discussing the risk associated with radiation-induced subsequent malignant neoplasms, the APRN explains that:', 'options': '{"A": risk increases with the amount of tissue treated., "B": malignancies usually occur within five years of treatment., "C": such malignancies are not likely., "D": the age of the radiation equipment matters.}', 'answer': 'risk increases with the amount of tissue treated.', 'answer_idx': 'A'}


In [11]:
def field_config(dataset):
    in_2 = {'role': 'user', 'content': f"""{dataset['question']} \n {dataset['options'].replace('{','').replace('}','')}""" }
    return {'input': [in_2], 
            'ideal': f"""{dataset['answer_idx']}"""
           }
dataset = dataset.map(field_config)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
dataset = dataset.remove_columns(['question', 'options', 'answer', 'answer_idx'])

In [13]:
dataset[0]

{'input': [{'content': 'When discussing the risk associated with radiation-induced subsequent malignant neoplasms, the APRN explains that: \n "A": risk increases with the amount of tissue treated., "B": malignancies usually occur within five years of treatment., "C": such malignancies are not likely., "D": the age of the radiation equipment matters.',
   'role': 'user'}],
 'ideal': 'A'}

In [14]:
import json

with open('output.jsonl', 'w') as outfile:
    for entry in dataset:
        json.dump(entry, outfile)
        outfile.write('\n')